In [1]:
import torch
x=torch.tensor(3.0)
y=torch.tensor(2.0)
print(x+y,x*y,x-y,x/y,x**y)

tensor(5.) tensor(6.) tensor(1.) tensor(1.5000) tensor(9.)


In [3]:
x=torch.arange(3)
x

tensor([0, 1, 2])

In [4]:
A=torch.arange(6,dtype=torch.float32).reshape(2,3)
B=A.clone()
A,A+B

(tensor([[0., 1., 2.],
         [3., 4., 5.]]),
 tensor([[ 0.,  2.,  4.],
         [ 6.,  8., 10.]]))

In [5]:
a=2
X=torch.arange(24).reshape(2,3,4)
a+X,(a+X).shape

(tensor([[[ 2,  3,  4,  5],
          [ 6,  7,  8,  9],
          [10, 11, 12, 13]],
 
         [[14, 15, 16, 17],
          [18, 19, 20, 21],
          [22, 23, 24, 25]]]),
 torch.Size([2, 3, 4]))

In [8]:
A.shape,A.sum()

(torch.Size([2, 3]), tensor(15.))

In [9]:
A.sum(axis=0).shape

torch.Size([3])

In [12]:
A.sum(axis=[0,1])==A.sum()

tensor(True)

In [13]:
sum_A=A.sum(axis=1,keepdim=True)
sum_A,sum_A.shape

(tensor([[ 3.],
         [12.]]),
 torch.Size([2, 1]))

In [14]:
A/sum_A

tensor([[0.0000, 0.3333, 0.6667],
        [0.2500, 0.3333, 0.4167]])

In [15]:
A.cumsum(axis=0)

tensor([[0., 1., 2.],
        [3., 5., 7.]])

In [23]:
a=torch.arange(9).reshape(3,3)
print(a)
(a.T).T==a


tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])


tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

In [24]:
b=torch.arange(9).reshape(3,3)
b=b.T
b 
d=a.T+b.T
c=(a+b).T
d==c

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

In [25]:
A.shape,A,A.sum(axis=1),A/A.sum(axis=1,keepdim=True)

(torch.Size([2, 3]),
 tensor([[0., 1., 2.],
         [3., 4., 5.]]),
 tensor([ 3., 12.]),
 tensor([[0.0000, 0.3333, 0.6667],
         [0.2500, 0.3333, 0.4167]]))

In [27]:
x=torch.tensor(4.0)
x.requires_grad_(True)
x.grad
x.grad=4*x
y=2*torch.dot(x,x)
y.backward()
x.grad


RuntimeError: 1D tensors expected, but got 0D and 0D tensors

In [1]:
import torch


In [3]:
x=torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [5]:
x.requires_grad_(True)
x.grad

In [9]:
y=2*torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

In [10]:
y.backward()
x.grad

tensor([ 0.,  8., 16., 24.])

In [11]:
x=torch.arange(4.0,requires_grad=True)
y=2*torch.dot(x,x)
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [12]:
y.backward()
x.grad

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [14]:
def f(a):
    b=a*2
    while b.norm()<1000:
        b=b*2
    if b.sum()>0:
        c=b
    else:
        c=100*b
    return c
a=torch.randn((4),requires_grad=True)
d=f(a)
d.backward()

RuntimeError: grad can be implicitly created only for scalar outputs

In [16]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [17]:
true_w=torch.tensor([2,-3.4])
true_b=4.2
features,labels=d2l.synthetic_data(true_w,true_b,1000)

In [18]:
def load_array(data_arrays,batch_size,is_train=True):
    dataset=data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)
batch_size=10
data_iter=load_array((features,labels),batch_size)

In [19]:
next(iter(data_iter))

[tensor([[ 1.3960, -0.2493],
         [ 0.9645, -0.2682],
         [ 0.0853,  1.8034],
         [-1.3912,  0.2452],
         [-0.5295,  1.1712],
         [-1.3908, -1.0408],
         [-0.8703, -0.2913],
         [ 0.0836,  1.4335],
         [-1.1805,  1.2090],
         [ 0.1677,  0.9390]]),
 tensor([[ 7.8368],
         [ 7.0328],
         [-1.7626],
         [ 0.5823],
         [-0.8214],
         [ 4.9525],
         [ 3.4597],
         [-0.5047],
         [-2.2574],
         [ 1.3319]])]

In [20]:
from torch import nn
net=nn.Sequential(nn.Linear(2,1))
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [21]:
loss=nn.MSELoss()

In [25]:
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

In [26]:
num_epochs=3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l=loss(net(X),y)
        trainer.zero_grad()
        l.sum().backward()
        trainer.step()
    l=loss(net(features),labels)
    print(f'epoch {epoch+1}, loss {l:f}')

epoch 1, loss 0.000483
epoch 2, loss 0.000105
epoch 3, loss 0.000104


In [27]:
w=net[0].weight.data
print('w_loss:',true_w-w.reshape(true_w.shape))
b=net[0].bias.data
print('b_loss:',true_b-b)

w_loss: tensor([-0.0004, -0.0003])
b_loss: tensor([0.0003])


In [1]:
from IPython import display